In [ ]:
# Force Calculation
import numpy as np
import matplotlib.pyplot as plt
from pylablib.devices import Thorlabs
import cv2
import math
from scipy.ndimage import gaussian_filter

# Initialize the camera
cam1 = Thorlabs.ThorlabsTLCamera()
Thorlabs.list_cameras_tlcam()

# Calculate force using the relation between displacement and time
def calculate_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Filter
def Gaussian_smooth(data, sigma):
    return gaussian_filter(data, sigma = sigma)

# Filter
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode = 'valid')

speed = []
distance = []

# Initialize Tracker
tracker = cv2.TrackerCSRT_create()

# Setup camera parameters and acquisition
cam1.set_exposure(0.03) # Example exposure setting
cam1.setup_acquisition(nframes=100)
cam1.start_acquisition()

cam1.wait_for_frame()
# Read the first frame to initialize the tracker
first_frame = cam1.read_newest_image()
if first_frame is not None:
# Convert the frame to an appropriate format for display
    scale_factor = 1 / np.max(first_frame)
    first_frame_display = (first_frame[:, :, (2, 1, 0)] * scale_factor * 255).astype(np.uint8)
    bbox = cv2.selectROI('Frame', first_frame_display, False) # Let user select ROI
    tracker.init(first_frame_display, bbox)
    last_center = (bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2)
    cv2.destroyAllWindows()
else:
    print("ERROR: Could not read the first frame.")

# Start tracking
while cam1.is_opened():
    cam1.wait_for_frame()
    frame = cam1.read_newest_image()
    if frame is not None:
# Convert the frame for display and tracking
        scale_factor = 1 / np.max(frame)
        frame_display = cv2.UMat((frame[:, :, (2, 1, 0)] * scale_factor * 255).astype(np.uint8))
# Update tracker and get the new position of the object
        success, bbox = tracker.update(frame_display)
        if success:
# Draw tracking result
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame_display, p1, p2, (255, 0, 0), 2, 1)
            current_center = (bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2)
            distance_moved = calculate_distance(last_center, current_center)
            speed_pixels_per_frame = distance_moved * 3.45 * 30 #Since time between frames is 1 frame
            distance_real = distance_moved * 3.45
            distance.append(distance_real)
            speed.append(speed_pixels_per_frame)
            last_center = current_center

# Display the frame with tracking result
        cv2.imshow('Frame', frame_display)
    else:
        print("ERROR: Could not read frame.")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam1.stop_acquisition()
cam1.close()

cv2.destroyAllWindows()




time = np.linspace(0, len(distance) - 1, len(distance)) * 1/30
displacements = distance - np.mean(distance)
fft_result = np.fft.fft(displacements)
frequencies = np.fft.fftfreq(fft_result.size, d = time[1]-time[0])

power_spectrum = np.abs(fft_result) ** 2
plt.figure(figsize=(10,6))
plt.loglog(frequencies[:len(frequencies)//2], power_spectrum[:len(power_spectrum)//2])
plt.title('Power Spectrum')
plt.xlabel('Frequency(Hz)')
plt.ylabel('Power($\mu m^2/s^2$)')
plt.ylim(100, 10 ** 8)
plt.grid(True)
plt.show()

# Curve Fitting
from scipy.optimize import curve_fit
def fitC(x, a, b, fc):
    return a / (b * (x + fc) ** 2)
y = power_spectrum[:len(power_spectrum)//2]
x = frequencies[:len(frequencies)//2]
FitPower,_ = curve_fit(fitC, x[0:17], y[0:17])
fig, ax = plt.subplots()

ax.loglog(x, y, 'b-', label='PowerSpectrum')
ax.loglog(x, fitC(x, *FitPower), 'r-', label='fit: a = %5.3f, b = %5.3f, fc = %5.3f' % tuple(FitPower))
ax.set_ylim(10, 10 ** 6)
ax.set_title('Power Spectrum')
ax.set_xlabel('Frequency(Hz)')
ax.set_ylabel('Power($\mu m^2/$)')
ax.legend()
ax.grid(True)
plt.savefig('PowerSpectrum.png')
